Data uploading

In [1]:
from google.colab import files
uploaded = files.upload()

Saving biodeg.csv to biodeg.csv


Library loading

In [2]:
!pip install deap

     |████████████████████████████████| 163kB 25.5MB/s 


In [57]:
import pandas as pd
import numpy as np
import random as rnd
import io

from deap import base
from deap import creator
from deap import tools

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.pipeline import make_pipeline

In [4]:
df = pd.read_csv(io.BytesIO(uploaded['biodeg.csv']), sep=';')
df_name = df.columns
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1055 entries, 0 to 1054
Data columns (total 42 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   SpMax_L             1055 non-null   float64
 1   J_Dz(e)             1055 non-null   float64
 2   nHM                 1055 non-null   int64  
 3   F01[N-N]            1055 non-null   int64  
 4   F04[C-N]            1055 non-null   int64  
 5   NssssC              1055 non-null   int64  
 6   nCb-                1055 non-null   int64  
 7   C%                  1055 non-null   float64
 8   nCp                 1055 non-null   int64  
 9   nO                  1055 non-null   int64  
 10  F03[C-N]            1055 non-null   int64  
 11  SdssC               1055 non-null   float64
 12  HyWi_B(m)           1055 non-null   float64
 13  LOC                 1055 non-null   float64
 14  SM6_L               1055 non-null   float64
 15  F03[C-O]            1055 non-null   int64  
 16  Me    

In [5]:
SEED = 42
np.random.seed(SEED)

### Genetic Algorithm

In [6]:
columns = df.drop('experimental_class', axis=1).columns
training = pd.concat([df])

1. Logistic Regression

In [7]:
def getPenaltyLR():
  penalties = ['l1', 'l2']
  index = rnd.randint(0, len(penalties) - 1)   
  penalty = penalties[index]

  return penalty

def getCLR():
  r = rnd.random()
  r2 = rnd.randint(0, 4)
  C = r + r2 + 0.000000001 

  return C

def getMultiClassLR():
  multi_classes = ['auto', 'ovr', 'multinomial']
  index = rnd.randint(0, len(multi_classes) - 1)   
  multi_class = multi_classes[index]

  return multi_class

In [40]:
def getModelLR(individual):
  penalty = individual[0]
  C = individual[1]
  multi_class = individual[2]

  model = LogisticRegression(penalty=penalty, C=C, multi_class=multi_class)

  return model

def getXyLR(individual):
  selected_columns = list(columns)

  for i in range(len(individual[3:])):
    if individual[3 + i] < 1:
      selected_columns.remove(columns[i])

  target_columns = np.append(['experimental_class'], selected_columns)
  df = training.loc[:, target_columns].dropna()
  X  = df.loc[:, selected_columns]
  y = np.ravel(df.loc[:, ['experimental_class']])
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
  standard_scaler = preprocessing.StandardScaler().fit(X)
  X_train = standard_scaler.transform(X_train)
  X_test = standard_scaler.transform(X_test)
  return [X_train, y_train, X_test, y_test, selected_columns, standard_scaler] 

def evaluteOneMaxLR(individual):
  model = getModelLR(individual)
  Xy = getXyLR(individual)
  scores = cross_val_score(model, Xy[0], Xy[1], cv=5, n_jobs=-1)
  mean = scores.mean(),
  
  return mean

def myMutateLR(individual, indpb=0.05):
  if rnd.random() < indpb:
      individual[0] = toolbox.attribute_PenaltyLR()

  if rnd.random() < indpb:
      individual[1] = toolbox.attribute_CLR()

  if rnd.random() < indpb:
      individual[2] = toolbox.attribute_Multi_ClassLR()
      
  for i in range(len(individual[3:])):
      if rnd.random() < indpb:
          individual[3 + i] = toolbox.attribute_bool()

In [ ]:
creator.create('FitnessMax', base.Fitness, weights=(1.0,))
creator.create('Individual', list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register('attribute_bool', rnd.randint, 0, 1)
toolbox.register('attribute_PenaltyLR', getPenaltyLR)
toolbox.register('attribute_CLR', getCLR)
toolbox.register('attribute_Multi_ClassLR', getMultiClassLR)

function_sequence = [toolbox.attribute_PenaltyLR, toolbox.attribute_CLR, toolbox.attribute_Multi_ClassLR]

for column in columns:
  function_sequence.append(toolbox.attribute_bool)

toolbox.register('individual', tools.initCycle, creator.Individual, function_sequence, 1)
toolbox.register('population', tools.initRepeat, list, toolbox.individual)

In [48]:
CXPB, MUTPB, NGEN, POPSIZE = 0.5, 0.2, 100, 100
pop = toolbox.population(n=POPSIZE) 

In [49]:
toolbox.register('evaluate', evaluteOneMaxLR)
toolbox.register('mate', tools.cxTwoPoint)
toolbox.register('mutate', myMutateLR, indpb=0.15)
toolbox.register('select', tools.selTournament, tournsize=3)
rnd.seed(66)

In [53]:
print('Start evolution of Logistic Regression')

fitnesses = list(map(toolbox.evaluate, pop))
for individual, fitness in zip(pop, fitnesses):
  individual.fitness.values = fitness

print (f'  Evaluated {len(pop)} individuals')

for generation in range(NGEN):
  print(f'-- Generation {generation} --')

  offspring = toolbox.select(pop, len(pop))
  offspring = list(map(toolbox.clone, offspring))

  for child1, child2 in zip(offspring[::2], offspring[1::2]):
    if rnd.random() < CXPB:
      c1 = toolbox.clone(child1)
      c2 = toolbox.clone(child2)
      toolbox.mate(child1, child2)
      if c1 != child1: 
        del child1.fitness.values
      if c2 != child2: 
        del child2.fitness.values

  for mutant in offspring:
    if rnd.random() < MUTPB:
      m1 = toolbox.clone(mutant)
      toolbox.mutate(mutant)
      if m1 != mutant: 
        del mutant.fitness.values

  invalid_individual = [individual for individual in offspring if not individual.fitness.valid]
  fitnesses = map(toolbox.evaluate, invalid_individual)

  for individual, fitness in zip(invalid_individual, fitnesses):
    individual.fitness.values = fitness

  print (f'  Evaluated {len(invalid_individual)} individuals')

  pop[:] = offspring
  fits = [individual.fitness.values[0] for individual in pop]

  length = len(pop)
  mean = sum(fits) / length
  sum2 = sum(x * x for x in fits)
  std = abs(sum2 / length - mean**2)**0.5
  best_individual = tools.selBest(pop, POPSIZE)[0]
  print(f'Best individual is {best_individual}, {best_individual.fitness.values}')        
  print(f'  Min: {min(fits)}')
  print(f'  Max: {max(fits)}')
  print(f'  Avg: {mean}')
  print(f'  Std: {std}')

print('End evolution of Logistic Regression')

best_individual = tools.selBest(pop, POPSIZE)[0]
print(f'Best individual is {best_individual}, {best_individual.fitness.values}')

Start evolution of Logistic Regression
  Evaluated 100 individuals
-- Generation 0 --
  Evaluated 47 individuals
Best individual is ['l1', 1.1018507432259939, 'auto', 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1], (nan,)
  Min: nan
  Max: nan
  Avg: nan
  Std: nan
-- Generation 1 --
  Evaluated 56 individuals
Best individual is ['l1', 0.2231408409893099, 'multinomial', 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0], (nan,)
  Min: nan
  Max: nan
  Avg: nan
  Std: nan
-- Generation 2 --
  Evaluated 60 individuals
Best individual is ['l1', 1.7396146756565254, 'auto', 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1], (nan,)
  Min: nan
  Max: nan
  Avg: nan
  Std: nan
-- Generation 3 --
  Evaluated 53 individuals
Best individual is ['l1', 1.1107541763601811, 'auto', 1

In [54]:
model = getModelLR(best_individual)
Xy = getXyLR(best_individual)
LR_columns = Xy[4]
scaler = Xy[5]
print(f'Selected Features: {LR_columns}')

Selected Features: ['J_Dz(e)', 'nHM', 'F01[N-N]', 'NssssC', 'C%', 'nO', 'HyWi_B(m)', 'LOC', 'Me', 'Mi', 'nN-N', 'nArNO2', 'SpPosA_B(p)', 'B01[C-Br]', 'B03[C-Cl]', 'N-073', 'SpMax_A', 'SdO', 'C-026', 'SpMax_B(m)', 'Psi_i_A', 'nN', 'SM6_B(m)', 'nArCOOR', 'nX']


In [59]:
X_train = Xy[0]
Y_train = Xy[1]
X_test = Xy[2]
Y_test = Xy[3]

scores = cross_val_score(model, X_train, Y_train, cv=5).mean()
cv_LRGA_score = scores.mean()
print(f'LRGA CV score: {scores.mean()}')
model.fit(X_train ,Y_train)
print(f'Training score: {model.score(X_train, Y_train)}')
prediction = model.predict(X_test)
print(f'Prediction score: {(prediction == Y_test).mean()}')
print('\n' + classification_report(Y_test, prediction))

LRGA CV score: 0.8482694772824384
Training score: 0.8718381112984823
Prediction score: 0.8939393939393939

              precision    recall  f1-score   support

         NRB       0.94      0.90      0.92       129
          RB       0.82      0.88      0.85        69

    accuracy                           0.89       198
   macro avg       0.88      0.89      0.89       198
weighted avg       0.90      0.89      0.89       198



2. K-Nearest Neighbor (KNN)

In [98]:
def getNeighborKNN():
  neighbor = rnd.randint(1, 20)

  return neighbor

def getWeightKNN():
  weights = ['uniform', 'distance']
  index = rnd.randint(0, len(weights) - 1)   
  weight = weights[index]

  return weight

In [99]:
def getModelKNN(individual):
  n_neighbors = individual[0]
  weights = individual[1]

  model = KNeighborsClassifier(n_neighbors=n_neighbors, weights=weights)

  return model

def getXyKNN(individual):
  selected_columns = list(columns)

  for i in range(len(individual[2:])):
    if individual[2 + i] < 1:
      selected_columns.remove(columns[i])

  target_columns = np.append(['experimental_class'], selected_columns)
  df = training.loc[:, target_columns].dropna()
  X  = df.loc[:, selected_columns]
  y = np.ravel(df.loc[:, ['experimental_class']])
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
  standard_scaler = preprocessing.StandardScaler().fit(X)
  X_train = standard_scaler.transform(X_train)
  X_test = standard_scaler.transform(X_test)
  return [X_train, y_train, X_test, y_test, selected_columns, standard_scaler] 

def evaluteOneMaxKNN(individual):
  model = getModelKNN(individual)
  Xy = getXyKNN(individual)
  scores = cross_val_score(model, Xy[0], Xy[1], cv=5, n_jobs=-1)
  mean = scores.mean(),
  
  return mean

def myMutateKNN(individual, indpb=0.05):
  if rnd.random() < indpb:
      individual[0] = toolbox.attribute_N_NeighborsKNN()

  if rnd.random() < indpb:
      individual[1] = toolbox.attribute_WeightsKNN()
      
  for i in range(len(individual[2:])):
      if rnd.random() < indpb:
          individual[2 + i] = toolbox.attribute_bool()

In [100]:
creator.create('FitnessMax', base.Fitness, weights=(1.0,))
creator.create('Individual', list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register('attribute_bool', rnd.randint, 0, 1)
toolbox.register('attribute_N_NeighborsKNN', getNeighborKNN)
toolbox.register('attribute_WeightsKNN', getWeightKNN)

function_sequence = [toolbox.attribute_N_NeighborsKNN, toolbox.attribute_WeightsKNN]

for column in columns:
  function_sequence.append(toolbox.attribute_bool)

toolbox.register('individual', tools.initCycle, creator.Individual, function_sequence, 1)
toolbox.register('population', tools.initRepeat, list, toolbox.individual)

/usr/local/lib/python3.7/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/usr/local/lib/python3.7/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


In [101]:
toolbox.register('evaluate', evaluteOneMaxKNN)
toolbox.register('mate', tools.cxTwoPoint)
toolbox.register('mutate', myMutateKNN, indpb=0.15)
toolbox.register('select', tools.selTournament, tournsize=3)
rnd.seed(66)

In [102]:
CXPB, MUTPB, NGEN, POPSIZE = 0.5, 0.2, 100, 100
pop = toolbox.population(n=POPSIZE) 

In [103]:
print('Start evolution of K-Nearest Neighbor')

fitnesses = list(map(toolbox.evaluate, pop))
for individual, fitness in zip(pop, fitnesses):
  individual.fitness.values = fitness

print (f'  Evaluated {len(pop)} individuals')

for generation in range(NGEN):
  print(f'-- Generation {generation} --')

  offspring = toolbox.select(pop, len(pop))
  offspring = list(map(toolbox.clone, offspring))

  for child1, child2 in zip(offspring[::2], offspring[1::2]):
    if rnd.random() < CXPB:
      c1 = toolbox.clone(child1)
      c2 = toolbox.clone(child2)
      toolbox.mate(child1, child2)
      if c1 != child1: 
        del child1.fitness.values
      if c2 != child2: 
        del child2.fitness.values

  for mutant in offspring:
    if rnd.random() < MUTPB:
      m1 = toolbox.clone(mutant)
      toolbox.mutate(mutant)
      if m1 != mutant: 
        del mutant.fitness.values

  invalid_individual = [individual for individual in offspring if not individual.fitness.valid]
  fitnesses = map(toolbox.evaluate, invalid_individual)

  for individual, fitness in zip(invalid_individual, fitnesses):
    individual.fitness.values = fitness

  print (f'  Evaluated {len(invalid_individual)} individuals')

  pop[:] = offspring
  fits = [individual.fitness.values[0] for individual in pop]

  length = len(pop)
  mean = sum(fits) / length
  sum2 = sum(x * x for x in fits)
  std = abs(sum2 / length - mean**2)**0.5
  best_individual = tools.selBest(pop, POPSIZE)[0]
  print(f'Best individual is {best_individual}, {best_individual.fitness.values}')        
  print(f'  Min: {min(fits)}')
  print(f'  Max: {max(fits)}')
  print(f'  Avg: {mean}')
  print(f'  Std: {std}')

print('End evolution of K-Nearest Neighbor')

best_individual = tools.selBest(pop, POPSIZE)[0]
print(f'Best individual is {best_individual}, {best_individual.fitness.values}')

Start evolution of K-Nearest Neighbor
  Evaluated 100 individuals
-- Generation 0 --
  Evaluated 53 individuals
Best individual is [5, 'uniform', 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1], (0.8583677538812134,)
  Min: 0.7875231448511608
  Max: 0.8583677538812134
  Avg: 0.8322539524284283
  Std: 0.012712791862626253
-- Generation 1 --
  Evaluated 48 individuals
Best individual is [4, 'distance', 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1], (0.8668423301524,)
  Min: 0.7993305796895029
  Max: 0.8668423301524
  Avg: 0.8394529269334849
  Std: 0.010705306432245725
-- Generation 2 --
  Evaluated 51 individuals
Best individual is [4, 'distance', 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1], (0.868494516450648,)
  Min: 0.8128329297820823
  Max: 0.86849451645064

In [104]:
model = getModelKNN(best_individual)
Xy = getXyKNN(best_individual)
KNN_columns = Xy[4]
scaler = Xy[5]
print(f'Selected Features: {KNN_columns}')

Selected Features: ['SpMax_L', 'nHM', 'F04[C-N]', 'NssssC', 'nCb-', 'nCp', 'SdssC', 'LOC', 'F03[C-O]', 'Me', 'nN-N', 'nArNO2', 'SpPosA_B(p)', 'B01[C-Br]', 'SpMax_A', 'TI2_L', 'nCrt', 'C-026', 'F02[C-N]', 'Psi_i_A', 'nN', 'SM6_B(m)', 'nArCOOR', 'nX']


In [105]:
X_train = Xy[0]
Y_train = Xy[1]
X_test = Xy[2]
Y_test = Xy[3]

scores = cross_val_score(model, X_train, Y_train, cv=5).mean()
cv_KNNGA_score = scores.mean()
print(f'KNNGA CV score: {scores.mean()}')
model.fit(X_train ,Y_train)
print(f'Training score: {model.score(X_train, Y_train)}')
prediction = model.predict(X_test)
print(f'Prediction score: {(prediction == Y_test).mean()}')
print('\n' + classification_report(Y_test, prediction))

KNNGA CV score: 0.8937758154109101
Training score: 1.0
Prediction score: 0.8686868686868687

              precision    recall  f1-score   support

         NRB       0.94      0.85      0.89       129
          RB       0.77      0.90      0.83        69

    accuracy                           0.87       198
   macro avg       0.85      0.88      0.86       198
weighted avg       0.88      0.87      0.87       198



3. Support Vector Machine (SVM)

In [71]:
def getCSVM():
  r = rnd.random()
  r2 = rnd.randint(0, 2)
  C = r + r2 + 0.000000001 

  return C

def getGammaSVM():
  r = rnd.random()
  r2 = rnd.randint(0, 3)
  gamma = 0.000000001 + (r / (10**r2))

  return gamma

def getKernelSVM():
  kernels = ['rbf', 'linear', 'sigmoid']
  index = rnd.randint(0, len(kernels) - 1)   
  kernel = kernels[index]

  return kernel

In [72]:
def getModelSVM(individual):
  C = individual[0]
  gamma = individual[1]
  kernel = individual[2]
  
  model = SVC(kernel=kernel, C=C, gamma=gamma)

  return model

def getXySVM(individual):
  selected_columns = list(columns)

  for i in range(len(individual[3:])):
    if individual[3 + i] < 1:
      selected_columns.remove(columns[i])

  target_columns = np.append(['experimental_class'], selected_columns)
  df = training.loc[:, target_columns].dropna()
  X  = df.loc[:, selected_columns]
  y = np.ravel(df.loc[:, ['experimental_class']])
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
  standard_scaler = preprocessing.StandardScaler().fit(X)
  X_train = standard_scaler.transform(X_train)
  X_test = standard_scaler.transform(X_test)
  return [X_train, y_train, X_test, y_test, selected_columns, standard_scaler] 

def evaluteOneMaxSVM(individual):
  model = getModelSVM(individual)
  Xy = getXySVM(individual)
  scores = cross_val_score(model, Xy[0], Xy[1], cv=5, n_jobs=-1)
  mean = scores.mean(),
  
  return mean

def myMutateSVM(individual, indpb=0.05):
  if rnd.random() < indpb:
      individual[0] = toolbox.attribute_CKNN()

  if rnd.random() < indpb:
      individual[1] = toolbox.attribute_GammaKNN()

  if rnd.random() < indpb:
      individual[2] = toolbox.attribute_KernelSVM()
      
  for i in range(len(individual[3:])):
      if rnd.random() < indpb:
          individual[3 + i] = toolbox.attribute_bool()

In [ ]:
creator.create('FitnessMax', base.Fitness, weights=(1.0,))
creator.create('Individual', list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register('attribute_bool', rnd.randint, 0, 1)
toolbox.register('attribute_CKNN', getCSVM)
toolbox.register('attribute_GammaKNN', getGammaSVM)
toolbox.register('attribute_KernelSVM', getKernelSVM)

function_sequence = [toolbox.attribute_CKNN, toolbox.attribute_GammaKNN, toolbox.attribute_KernelSVM]

for column in columns:
  function_sequence.append(toolbox.attribute_bool)

toolbox.register('individual', tools.initCycle, creator.Individual, function_sequence, 1)
toolbox.register('population', tools.initRepeat, list, toolbox.individual)

In [74]:
toolbox.register('evaluate', evaluteOneMaxSVM)
toolbox.register('mate', tools.cxTwoPoint)
toolbox.register('mutate', myMutateSVM, indpb=0.15)
toolbox.register('select', tools.selTournament, tournsize=3)
rnd.seed(66)

In [75]:
CXPB, MUTPB, NGEN, POPSIZE = 0.5, 0.2, 100, 100
pop = toolbox.population(n=POPSIZE) 

In [76]:
print('Start evolution of Support Vector Machine')

fitnesses = list(map(toolbox.evaluate, pop))
for individual, fitness in zip(pop, fitnesses):
  individual.fitness.values = fitness

print (f'  Evaluated {len(pop)} individuals')

for generation in range(NGEN):
  print(f'-- Generation {generation} --')

  offspring = toolbox.select(pop, len(pop))
  offspring = list(map(toolbox.clone, offspring))

  for child1, child2 in zip(offspring[::2], offspring[1::2]):
    if rnd.random() < CXPB:
      c1 = toolbox.clone(child1)
      c2 = toolbox.clone(child2)
      toolbox.mate(child1, child2)
      if c1 != child1: 
        del child1.fitness.values
      if c2 != child2: 
        del child2.fitness.values

  for mutant in offspring:
    if rnd.random() < MUTPB:
      m1 = toolbox.clone(mutant)
      toolbox.mutate(mutant)
      if m1 != mutant: 
        del mutant.fitness.values

  invalid_individual = [individual for individual in offspring if not individual.fitness.valid]
  fitnesses = map(toolbox.evaluate, invalid_individual)

  for individual, fitness in zip(invalid_individual, fitnesses):
    individual.fitness.values = fitness

  print (f'  Evaluated {len(invalid_individual)} individuals')

  pop[:] = offspring
  fits = [individual.fitness.values[0] for individual in pop]

  length = len(pop)
  mean = sum(fits) / length
  sum2 = sum(x * x for x in fits)
  std = abs(sum2 / length - mean**2)**0.5
  best_individual = tools.selBest(pop, POPSIZE)[0]
  print(f'Best individual is {best_individual}, {best_individual.fitness.values}')        
  print(f'  Min: {min(fits)}')
  print(f'  Max: {max(fits)}')
  print(f'  Avg: {mean}')
  print(f'  Std: {std}')

print('End evolution of Support Vector Machine')

best_individual = tools.selBest(pop, POPSIZE)[0]
print(f'Best individual is {best_individual}, {best_individual.fitness.values}')

Start evolution of Support Vector Machine
  Evaluated 100 individuals
-- Generation 0 --
  Evaluated 57 individuals
Best individual is [2.3963869802196656, 0.08695081816031329, 'rbf', 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1], (0.8600769121207806,)
  Min: 0.6357214072069506
  Max: 0.8600769121207806
  Avg: 0.8266352371457052
  Std: 0.03516426154814159
-- Generation 1 --
  Evaluated 56 individuals
Best individual is [2.595940786625274, 0.03227389376834841, 'rbf', 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1], (0.8718416180031335,)
  Min: 0.8161942743198976
  Max: 0.8718416180031335
  Avg: 0.8462844324170345
  Std: 0.010899114995190922
-- Generation 2 --
  Evaluated 66 individuals
Best individual is [2.595940786625274, 0.03227389376834841, 'rbf', 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 

In [77]:
model = getModelSVM(best_individual)
Xy = getXySVM(best_individual)
SVM_columns = Xy[4]
scaler = Xy[5]
print(f'Selected Features: {SVM_columns}')

Selected Features: ['nHM', 'NssssC', 'nCb-', 'nCp', 'SdssC', 'HyWi_B(m)', 'LOC', 'SM6_L', 'Me', 'Mi', 'nN-N', 'nArNO2', 'nCRX3', 'SpPosA_B(p)', 'nCIR', 'B01[C-Br]', 'B03[C-Cl]', 'N-073', 'B04[C-Br]', 'TI2_L', 'nCrt', 'C-026', 'nHDon', 'Psi_i_A', 'nN', 'nArCOOR', 'nX']


In [78]:
X_train = Xy[0]
Y_train = Xy[1]
X_test = Xy[2]
Y_test = Xy[3]

scores = cross_val_score(model, X_train, Y_train, cv=5).mean()
cv_SVMGA_score = scores.mean()
print(f'SVMGA CV score: {scores.mean()}')
model.fit(X_train ,Y_train)
print(f'Training score: {model.score(X_train, Y_train)}')
prediction = model.predict(X_test)
print(f'Prediction score: {(prediction == Y_test).mean()}')
print('\n' + classification_report(Y_test, prediction))

SVMGA CV score: 0.8971656459193846
Training score: 0.954468802698145
Prediction score: 0.8686868686868687

              precision    recall  f1-score   support

         NRB       0.89      0.91      0.90       129
          RB       0.82      0.80      0.81        69

    accuracy                           0.87       198
   macro avg       0.86      0.85      0.85       198
weighted avg       0.87      0.87      0.87       198



4. Decision Tree

In [79]:
def getMaxDepthDT():
  max_depth = rnd.randint(3, 4)
  return max_depth

def getMaxFeaturesDT():
  max_features = rnd.randint(1, 4)
  return max_features

def getMinSamplesLeafDT():
  min_samples_leaf = rnd.randint(1, 4)
  return min_samples_leaf

def getMinSamplesSplitDT():
  min_samples_split = rnd.randint(1, 20)
  return min_samples_split

def getCriterionDT():
  criteria = ['gini', 'entropy']
  index = rnd.randint(0, len(criteria) - 1)   
  criterion = criteria[index]

  return criterion

In [80]:
def getModelDT(individual):
  max_depth = individual[0]
  max_features = individual[1]
  min_samples_leaf = individual[2]
  min_samples_split = individual[3]
  criterion = individual[4]
  
  model = DecisionTreeClassifier(max_depth=max_depth, max_features=max_features, min_samples_leaf=min_samples_leaf, min_samples_split=min_samples_split, criterion=criterion)

  return model

def getXyDT(individual):
  selected_columns = list(columns)

  for i in range(len(individual[5:])):
    if individual[5 + i] < 1:
      selected_columns.remove(columns[i])

  target_columns = np.append(['experimental_class'], selected_columns)
  df = training.loc[:, target_columns].dropna()
  X  = df.loc[:, selected_columns]
  y = np.ravel(df.loc[:, ['experimental_class']])
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
  standard_scaler = preprocessing.StandardScaler().fit(X)
  X_train = standard_scaler.transform(X_train)
  X_test = standard_scaler.transform(X_test)
  return [X_train, y_train, X_test, y_test, selected_columns, standard_scaler]  

def evaluteOneMaxDT(individual):
  model = getModelDT(individual)
  Xy = getXyDT(individual)
  scores = cross_val_score(model, Xy[0], Xy[1], cv=5, n_jobs=-1)
  mean = scores.mean(),
  
  return mean

def myMutateDT(individual, indpb=0.05):
  if rnd.random() < indpb:
      individual[0] = toolbox.attribute_Max_Depth_DT()

  if rnd.random() < indpb:
      individual[1] = toolbox.attribute_Max_FeaturesDT()

  if rnd.random() < indpb:
      individual[2] = toolbox.attribute_Min_Samples_LeafDT()
  
  if rnd.random() < indpb:
      individual[2] = toolbox.attribute_Min_Samples_SplitDT()

  if rnd.random() < indpb:
    individual[2] = toolbox.attribute_Min_Samples_SplitDT()
      
  for i in range(len(individual[5:])):
      if rnd.random() < indpb:
          individual[5 + i] = toolbox.attribute_bool()

In [81]:
creator.create('FitnessMax', base.Fitness, weights=(1.0,))
creator.create('Individual', list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register('attribute_bool', rnd.randint, 0, 1)
toolbox.register('attribute_Max_Depth_DT', getMaxDepthDT)
toolbox.register('attribute_Max_FeaturesDT', getMaxFeaturesDT)
toolbox.register('attribute_Min_Samples_LeafDT', getMinSamplesLeafDT)
toolbox.register('attribute_Min_Samples_SplitDT', getMinSamplesSplitDT)
toolbox.register('attribute_CriterionDT', getCriterionDT)

function_sequence = [toolbox.attribute_Max_Depth_DT, toolbox.attribute_Max_FeaturesDT, toolbox.attribute_Min_Samples_LeafDT, toolbox.attribute_Min_Samples_SplitDT, toolbox.attribute_CriterionDT]

for column in columns:
  function_sequence.append(toolbox.attribute_bool)

toolbox.register('individual', tools.initCycle, creator.Individual, function_sequence, 1)
toolbox.register('population', tools.initRepeat, list, toolbox.individual)

/usr/local/lib/python3.7/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/usr/local/lib/python3.7/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


In [82]:
toolbox.register('evaluate', evaluteOneMaxDT)
toolbox.register('mate', tools.cxTwoPoint)
toolbox.register('mutate', myMutateDT, indpb=0.15)
toolbox.register('select', tools.selTournament, tournsize=3)
rnd.seed(66)

In [83]:
CXPB, MUTPB, NGEN, POPSIZE = 0.5, 0.2, 100, 100
pop = toolbox.population(n=POPSIZE) 

In [84]:
print('Start evolution of Decision Tree')

fitnesses = list(map(toolbox.evaluate, pop))
for individual, fitness in zip(pop, fitnesses):
  individual.fitness.values = fitness

print (f'  Evaluated {len(pop)} individuals')

for generation in range(NGEN):
  print(f'-- Generation {generation} --')

  offspring = toolbox.select(pop, len(pop))
  offspring = list(map(toolbox.clone, offspring))

  for child1, child2 in zip(offspring[::2], offspring[1::2]):
    if rnd.random() < CXPB:
      c1 = toolbox.clone(child1)
      c2 = toolbox.clone(child2)
      toolbox.mate(child1, child2)
      if c1 != child1: 
        del child1.fitness.values
      if c2 != child2: 
        del child2.fitness.values

  for mutant in offspring:
    if rnd.random() < MUTPB:
      m1 = toolbox.clone(mutant)
      toolbox.mutate(mutant)
      if m1 != mutant: 
        del mutant.fitness.values

  invalid_individual = [individual for individual in offspring if not individual.fitness.valid]
  fitnesses = map(toolbox.evaluate, invalid_individual)

  for individual, fitness in zip(invalid_individual, fitnesses):
    individual.fitness.values = fitness

  print (f'  Evaluated {len(invalid_individual)} individuals')

  pop[:] = offspring
  fits = [individual.fitness.values[0] for individual in pop]

  length = len(pop)
  mean = sum(fits) / length
  sum2 = sum(x * x for x in fits)
  std = abs(sum2 / length - mean**2)**0.5
  best_individual = tools.selBest(pop, POPSIZE)[0]
  print(f'Best individual is {best_individual}, {best_individual.fitness.values}')        
  print(f'  Min: {min(fits)}')
  print(f'  Max: {max(fits)}')
  print(f'  Avg: {mean}')
  print(f'  Std: {std}')

print('End evolution of Decision Tree')

best_individual = tools.selBest(pop, POPSIZE)[0]
print(f'Best individual is {best_individual}, {best_individual.fitness.values}')

Start evolution of Decision Tree
  Evaluated 100 individuals
-- Generation 0 --
  Evaluated 53 individuals
Best individual is [4, 2, 2, 2, 'entropy', 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0], (0.7824526420737786,)
  Min: 0.681241988320752
  Max: 0.7977923372738925
  Avg: nan
  Std: nan
-- Generation 1 --
  Evaluated 60 individuals
Best individual is [4, 2, 3, 3, 'gini', 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1], (0.8010539809143996,)
  Min: 0.6980487110098277
  Max: 0.8010539809143996
  Avg: nan
  Std: nan
-- Generation 2 --
  Evaluated 49 individuals
Best individual is [4, 4, 2, 1, 'gini', 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1], (nan,)
  Min: 0.6980487110098277
  Max: 0.8010539809143996
  Avg: nan
  Std: nan
-- Generation 3 --
  Evaluated 59 

In [85]:
model = getModelDT(best_individual)
Xy = getXyDT(best_individual)
DT_columns = Xy[4]
scaler = Xy[5]
print(f'Selected Features: {DT_columns}')

Selected Features: ['SpMax_L', 'nHM', 'F01[N-N]', 'F04[C-N]', 'nCb-', 'C%', 'nCp', 'nO', 'F03[C-N]', 'SM6_L', 'F03[C-O]', 'nArNO2', 'B03[C-Cl]', 'Psi_i_1d', 'SdO', 'TI2_L', 'F02[C-N]', 'Psi_i_A', 'nN', 'SM6_B(m)', 'nArCOOR', 'nX']


In [86]:
X_train = Xy[0]
Y_train = Xy[1]
X_test = Xy[2]
Y_test = Xy[3]

scores = cross_val_score(model, X_train, Y_train, cv=5).mean()
cv_DTGA_score = scores.mean()
print(f'DTGA CV score: {scores.mean()}')
model.fit(X_train ,Y_train)
print(f'Training score: {model.score(X_train, Y_train)}')
prediction = model.predict(X_test)
print(f'Prediction score: {(prediction == Y_test).mean()}')
print('\n' + classification_report(Y_test, prediction))

DTGA CV score: 0.7470730665147416
Training score: 0.836424957841484
Prediction score: 0.8181818181818182

              precision    recall  f1-score   support

         NRB       0.84      0.88      0.86       129
          RB       0.76      0.70      0.73        69

    accuracy                           0.82       198
   macro avg       0.80      0.79      0.80       198
weighted avg       0.82      0.82      0.82       198

